# Modèle EOLES

Traduction en python du modèle EOLES créé par Philippe Quirion.

Notebook pour simulation du modèle

- Import des données 
- Définition des paramètres autres (eta_in, eta_out, etc) 
- Création du modèle Pyomo pour optimiser la répartition du stockage quand les vre ne répondent pas à la demande
- Output: Génération heure par heure par chaque tecno dans un excel 
- Etat actuel: ne fonctionne pas, problèmes sur les contraintes -> toutes les valeurs de l'excel sont à 0. 

**Pour run:** soit le solveur cbc est installé sur jupyter et tout va bien / soit il est installé ailleurs (genre un terminal ubuntu par exemple) et ducoup faut download as .py et run à partir du terminal. 

### Imports

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import csv
import time
import sys
# Imports pour pyomo
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

### Définition des variables

In [2]:
# Months 
m = ["jan","feb", "mar","apr","jun","jul","aug","sep","oct","nov","dec"]

# Technologies 
#tec = ["offshore","onshore","pv","river","lake","biogas","gas","phs","battery","methanation"]
tec = ["offshore","onshore","pv","phs","battery"] #modèle simplifié

# Power plants
# gen = ["offshore","onshore","pv","river","lake","biogas","gas"]
gen = ["offshore","onshore","pv"]

# Variable tec
vre = ["offshore","onshore","pv"]

# Non combustible generation tec
# ncomb = ["offshore","onshore","pv","river","lake","phs","battery"]
ncomb = ["offshore","onshore","pv","phs","battery"]

# Combustible generation tec
# comb = ["biogas","methanation"]

# Storage technologies
# stor = ["phs","battery","methanation"]
stor = ["phs","battery"]

# Technologies for upward FRR 
# frr = ["lake","phs","battery","gas"]
frr = ["phs","battery"]

### Input Data

Import des données. 

In [3]:
# profil des VRE par heure (éolien + PV)
load_factor = pd.read_csv("inputs/vre_profiles2006.csv", index_col=[0, 1], squeeze=True, header=None)

# demande d'électricité en 2050 par heure 
demand_2050 = pd.read_csv("inputs/demand2050_ademe.csv", header = None, names = ["h","demand_2050"])

# profil des lacs par mois 
lake_inflows = pd.read_csv("inputs/lake_inflows.csv", header = None , names = ["m", "lake_inflows"])

# profil des rivières par heure
gene_river = pd.read_csv("inputs/run_of_river.csv", header = None , names = ["h","gene_river"])

# additional FRR requirement for variable renewable energies because of forecast errors
epsilon = pd.read_csv("inputs/reserve_requirements.csv", header = None, names = ["vre","epsilon"])

# capacités existantes pour les technologies en GW
capa_ex = pd.read_csv("inputs/existing_capas.csv", header = None, names=["tec","capa_ex"])

# Capacité maximum par technologies en GW 
capa_max = pd.read_csv("inputs/max_capas.csv", header = None, names=["tec","capa_max"])

# annualized power capex cost in M€/GW/year 
capex = pd.read_csv("inputs/annuities.csv", header = None, names = ["tec", "capex"])

# annualized energy capex cost of storage technologies in M€/GWh/year'
capex_en = pd.read_csv("inputs/str_annuities.csv", header = None, names = ["str","capex"])

# annualized fixed operation and maintenance costs M€/GW/year
fOM = pd.read_csv("inputs/fO&M.csv", header = None, names=["tec", "fOM"])

# Variable operation and maintenance costs in M€/GWh
vOM = pd.read_csv("inputs/vO&M.csv", header = None, names = ["tec", "vOM"])

In [4]:
   # Demand profile in each our in GW
demand_2050 = pd.read_csv("inputs/demand2050_ademe.csv",index_col=0, squeeze=True, header=None)
    # Monthly lake inflows in GWh
lake_inflows = pd.read_csv("inputs/lake_inflows.csv", index_col=0, squeeze=True, header=None)
    # Additional FRR requirement for variable renewable energies because of forecast errors
epsilon = pd.read_csv("inputs/reserve_requirements.csv", index_col=0, squeeze=True, header=None)
    # Existing capacities of the technologies by December 2017 in GW
capa_ex = pd.read_csv("inputs/existing_capas.csv", index_col=0, squeeze=True, header=None)
    # Maximum capacities of the technologies in GW
capa_max = pd.read_csv("inputs/max_capas.csv", index_col=0, squeeze=True, header=None)
    # Annualized power capex cost in M€/GW/year
capex = pd.read_csv("inputs/annuities.csv", index_col=0, squeeze=True, header=None)
    # Annualized energy capex cost of storage technologies in M€/GWh/year
capex_en = pd.read_csv("inputs/str_annuities.csv", index_col=0, squeeze=True, header=None)
    # Annualized fixed operation and maintenance costs M€/GW/year
fOM = pd.read_csv("inputs/fO&M.csv", index_col=0, squeeze=True, header=None)
    # Variable operation and maintenance costs in M€/GWh
vOM = pd.read_csv("inputs/vO&M.csv", index_col=0, squeeze=True, header=None)

In [5]:
# charging related annuity of storage in M€/GW/year : PHS, battery, methanation 
s_capex = [0,0,84.16086]
# charging related fOM of storage in M€/GW/year
s_opex = [7.5,0,59.25]
# charging efifciency of storage technologies
eta_in = [0.95,0.9,0.59]
# discharging efficiency of storage technolgoies
eta_out = [0.9, 0.95, 0.45]
# pumping capacity in GW
pump_cap = 9.3
# maximum volume of energy can be stored in PHS reservoir in TWh
max_phs = 0.18
# maxium energy can be generated by biogas in TWh
max_biogas = 15
# uncertainty coefficient for hourly demand
load_uncertainty = 0.01
# load variation factor
delta = 0.1
# yearly fixed cost of each tec in M€/GW/year
fixed_costs = pd.DataFrame({'tec' : capex.index, 'fixed_costs' : capex + fOM})

## Simulation 

In [6]:
# Valeurs obtenues après run du modèle.py

# Capacity in GW
Q_tec = [[25.33, 120.0, 95.0, 7.2, 12.39 ]]
Q_tec = pd.DataFrame( Q_tec, columns = tec)


### A run 1 seule fois### 
eta_in = pd.DataFrame([[eta_in[0], eta_in[1]]], columns = stor)
eta_out = pd.DataFrame([[eta_out[0], eta_out[1]]], columns = stor)
s_opex = pd.DataFrame([[7.5, 1.96]], columns = stor)
s_capex = pd.DataFrame([[s_capex[0], s_capex[1]]], columns = stor)
### --- ###

# Energy volume of storage technology in GWh
Volume_str = [135, 49.55]

epsilon = epsilon.rename(index ={"PV": "pv"})
vOM = vOM.rename(index={"Onshore": "onshore", "PV":"pv", "PHS":"phs", "Battery":"battery"})

In [7]:
# Calcul de G_tec
# G_offshore = pd.DataFrame(load_factor[load_factor["vre"]=="offshore"]["load_factor"]*Q_tec[0])

# G_onshore = pd.DataFrame(load_factor[load_factor["vre"]=="onshore"]["load_factor"]*Q_tec[1])

# G_pv = pd.DataFrame(load_factor[load_factor["vre"]=="pv"]["load_factor"]*Q_tec[2])

# G_phs < Q_phs
# G_batterie < Q_batterie
# G_phs = 0
# G_battery = 0

# G_tec_h = {"G_offshore" : G_offshore, "G_onshore" : G_onshore, "G_pv" : G_pv, "G_phs" : G_phs, "G_battery" : G_battery}

In [8]:
# Initialisation du modèle Pyomo 
model = pyo.ConcreteModel()

In [9]:
# Définition des set 

first_hour = 0
last_hour = len(demand_2050)

#Range of hour in one year
model.h = pyo.RangeSet(first_hour,last_hour-1)
#Months
model.months = pyo.RangeSet(1,12)
#Technologies
model.tec = pyo.Set(initialize=tec)
#Power plants
model.gen = pyo.Set(initialize=gen)
#Variables Technologies
model.vre = pyo.Set(initialize=vre)

#
model.balance = pyo.Set(initialize=["offshore_f", "offshore_g", "onshore", "pv_g", "pv_c", "river", "lake","nuc","phs","battery1","battery4","h2_ccgt","ocgt","ccgt"])

#Storage Technologies
model.str = pyo.Set(initialize=stor)

#Storage Technologies
#model.str_noH2 = pyo.Set(initialize=["phs", "battery1","battery4"])

#Battery Storage
#model.battery = pyo.Set(initialize=["battery1","battery4"])

#Technologies for upward FRR
model.frr =pyo.Set(initialize=frr)

In [10]:
def s_bounds(model,i):
    if i == 'phs' :
        return (5.2,7.2)
    else :
        return (None,None)

In [11]:
# Définitions des variables
# Hourly energy generation in GWh/h
model.gene = pyo.Var(((tec, h) for tec in model.tec for h in model.h), within=pyo.NonNegativeReals,initialize=0)

# Hourly electricity input of battery storage GW
model.storage = pyo.Var(((storage, h) for storage in model.str for h in model.h), within=pyo.NonNegativeReals,initialize=0)

# Energy stored in each storage technology in GWh = Stage of charge
model.stored = pyo.Var(((storage, h) for storage in model.str for h in model.h), within=pyo.NonNegativeReals,initialize=0)

# Charging power capacity of each storage technology
model.s = pyo.Var(model.str, within=pyo.NonNegativeReals, initialize=0,bounds=s_bounds)

# Required upward frequency restoration reserve in GW    
model.reserve = pyo.Var(((reserve, h) for reserve in model.frr for h in model.h), within=pyo.NonNegativeReals,initialize=0)

In [12]:
"""CONSTRAINTS RULE

Set up a function which will return the equation of the constraint.
"""
print("- Définition des contraintes")
def generation_vre_constraint_rule(model, h, vre):
    """Get constraint on variables renewable profiles generation."""

    return model.gene[vre, h] == Q_tec[vre].values[0] * load_factor[vre,h]

def generation_capacity_constraint_rule(model, h, tec):
    """Get constraint on maximum power for non-VRE technologies."""

    return Q_tec[tec].values[0] >= model.gene[tec,h]


def frr_capacity_constraint_rule(model, h, frr):
    """Get constraint on maximum generation including reserves"""

    return Q_tec[frr].values[0] >= model.gene[frr, h] + model.reserve[frr, h]

def storing_constraint_rule(model, h, storage_tecs):
    """Get constraint on storing."""

    hPOne = h+1 if h<(last_hour-1) else 0
    charge = model.storage[storage_tecs, h] * eta_in[storage_tecs].values[0]
    discharge =  model.gene[storage_tecs, h] / eta_out[storage_tecs].values[0]
    flux = charge - discharge
    return model.stored[storage_tecs, hPOne] == model.stored[storage_tecs, h] + flux

def storage_constraint_rule(model,storage_tecs):
    """Get constraint on stored energy to be equal at the end than at the start."""

    first = model.stored[storage_tecs, first_hour]
    last = model.stored[storage_tecs, last_hour-1]
    charge = model.storage[storage_tecs, last_hour-1] * eta_in[storage_tecs].values[0]
    discharge = model.gene[storage_tecs, last_hour-1] / eta_out[storage_tecs].values[0]
    flux = charge - discharge
    return first == last + flux

def stored_capacity_constraint(model, h, storage_tecs):
    """Get constraint on maximum energy that is stored in storage units"""

    return model.stored[storage_tecs, h] <= Q_tec[storage_tecs].values[0]


def reserves_constraint_rule(model, h):
    """Get constraint on water for lake reservoirs."""

    res_req = sum(epsilon[vre] * Q_tec[vre].values[0] for vre in model.vre)
    load_req = demand_2050[h] *load_uncertainty * (1 + delta)
    return sum(model.reserve[frr, h] for frr in model.frr) ==  res_req + load_req

def adequacy_constraint_rule(model, h):
    """Get constraint for 'supply/demand relation'"""

    sto = sum(model.storage[stor, h] for stor in model.str)
    
    return sum(model.gene[tec, h] for tec in model.tec) >= (demand_2050[h] + sto )

def objective_rule(model):
    """Get constraint for the final objective function."""

    return (sum(model.s[storage_tecs] * (s_opex[storage_tecs].values[0] + s_capex[storage_tecs].values[0]) for storage_tecs in model.str)\
           + sum(sum(model.gene[tec, h] * vOM[tec] for h in model.h) for tec in model.tec))/1000


- Définition des contraintes


In [13]:
"""CONSTRAINT CREATION

Create the constraint as an object of the model with the function declared earlier as a rule.
"""

model.generation_vre_constraint = pyo.Constraint(model.h, model.vre, rule=generation_vre_constraint_rule)


# Pas utile pour le moment car que des énergies renouvelables
#model.generation_capacity_constraint = pyo.Constraint(model.h, model.tec, rule=generation_capacity_constraint_rule)

#model.battery_capacity_constraint = pyo.Constraint(rule=battery_capacity_constraint_rule)

#model.frr_capacity_constraint = pyo.Constraint(model.h, model.frr, rule=frr_capacity_constraint_rule)

# model.storing_constraint = pyo.Constraint(model.h,model.str, rule=storing_constraint_rule)

# model.storage_constraint = pyo.Constraint(model.str, rule=storage_constraint_rule)

# model.stored_capacity_constraint = pyo.Constraint(model.h, model.str, rule=stored_capacity_constraint)


# model.reserves_constraint = pyo.Constraint(model.h, rule=reserves_constraint_rule)

model.adequacy_constraint =  pyo.Constraint(model.h, rule=adequacy_constraint_rule)

#Creation of the objective -> Cost
model.objective = pyo.Objective(rule=objective_rule)

ERROR: Rule failed when generating expression for Objective objective with
    index None: AttributeError: 'Series' object has no attribute
    'is_expression_type'
ERROR: Constructing component 'objective' from data=None failed:
    AttributeError: 'Series' object has no attribute 'is_expression_type'


AttributeError: 'Series' object has no attribute 'is_expression_type'

In [ ]:
print("- Choix de l'optimiseur")
#opt = SolverFactory('gurobi')
opt = SolverFactory('cbc')

print("- Optimisation du modèle....") 
results = opt.solve(model)
#model.display()
print("- Optimisation terminée.")

In [ ]:
# outputs
print("- Production des sorties")
model_name = "simu_1"

In [ ]:
# Hourly_Generation

hourly_file = model_name + "_hourly_generation.csv"

with open(hourly_file,"w",newline="") as hourly:
        hourly_writer = csv.writer(hourly)

        hourly_header = ["hour"]
        for tec in model.tec:
            hourly_header.append(tec)
        
        for stor in model.str: 
            hourly_header.append("Storage " + stor)
            
        for stor in model.str: 
            hourly_header.append("Stored " + stor)
        
        for rsv in model.frr:
            hourly_header.append("frr " + rsv)
        hourly_header.append("Electricity demand")
        hourly_writer.writerow(hourly_header)

               
        for hour in model.h:
            hourly_data = [hour]
            
            # Génération
            for tec in model.tec:
                hourly_data.append(round(pyo.value(model.gene[tec,hour]),2))

            # Stockage
            for storage_tecs in model.str:
                hourly_data.append(round(pyo.value(model.storage[storage_tecs,hour]),2))
            
            # Stored
            for storage_tecs in model.str:
                hourly_data.append(round(pyo.value(model.stored[storage_tecs,hour]),2))
            
            # Reserve
            for frr in model.frr:
                hourly_data.append(round(pyo.value(model.reserve[frr,hour]),2))
            
            hourly_data.append(round(demand_2050[hour],2))
            hourly_writer.writerow(hourly_data)